<a href="https://colab.research.google.com/github/nikhil-mathews/MastersPr_Predicting-Human-Pathogen-PPIs-using-Natural-Language-Processing-methods/blob/main/3.Attention%20%26%20Transformers/att_BiLSTM_3D%2B4D%2B5D%2B6D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#Google colab does not have pickle
try:
  import pickle5 as pickle
except:
  !pip install pickle5
  import pickle5 as pickle
import os
import seaborn as sns
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten, Attention
from keras.layers import Conv1D, MaxPooling1D, Embedding,  Concatenate, Lambda,RepeatVector,Dot
from keras.models import Model
from sklearn.metrics import roc_auc_score,confusion_matrix,roc_curve, auc
from numpy import random
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

import sys
sys.path.insert(0,'/content/drive/MyDrive/ML_Data/')
import functions as f


def load_data(D=1,randomize=False):
    try:
        with open('/content/drive/MyDrive/ML_Data/df_train_'+str(D)+'D.pickle', 'rb') as handle:
            df_train = pickle.load(handle)
    except:
        df_train = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-train.pkl")
    try:
        with open('/content/drive/MyDrive/ML_Data/df_test_'+str(D)+'D.pickle', 'rb') as handle:
            df_test = pickle.load(handle)
    except:
        df_test = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-independent.pkl")
    if randomize:
        return shuff_together(df_train,df_test)
    else:
        return df_train,df_test

     |████████████████████████████████| 133kB 9.0MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219249 sha256=6dedbc5abd76d00a2dd06d4fca8c899839be53ddeea6cc087c914b4aa3a1f681
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
df_train,df_test = load_data(3)
print('The data used will be:')
df_train[['Human','Yersinia']]

The data used will be:


,Human,Yersinia
0,"[MKD, KDK, DKQ, KQK, QKK, KKK, KKK, KKE, KER, ...","[MAK, AKA, KAS, ASR, SRH, RHN, HNL, NLS, LSI, ..."
1,"[MSW, SWS, WSG, SGL, GLL, LLH, LHG, HGL, GLN, ...","[MQH, QHV, HVT, VTG, TGS, GSK, SKR, KRR, RRL, ..."
2,"[MSL, SLF, LFD, FDL, DLF, LFR, FRG, RGF, GFF, ...","[MAE, AEL, ELP, LPA, PAK, AKR, KRR, RRF, RFT, ..."
3,"[MAV, AVA, VAP, APR, PRL, RLF, LFG, FGG, GGL, ...","[MRI, RIF, IFA, FAI, AIS, ISC, SCS, CSS, SSY, ..."
4,"[MST, STI, TIQ, IQS, QSE, SET, ETD, TDC, DCY, ...","[MSY, SYA, YAF, AFP, FPG, PGT, GTF, TFP, FPG, ..."
...,...,...
6265,"[MSY, SYF, YFG, FGE, GEH, EHF, HFW, FWG, WGE, ...","[MIT, ITT, TTD, TDG, DGN, GNS, NSA, SAV, AVA, ..."
6266,"[MTV, TVG, VGK, GKS, KSS, SSK, SKM, KML, MLQ, ...","[MSQ, SQP, QPP, PPF, PFW, FWQ, WQQ, QQK, QKT, ..."
6267,"[MNN, NNL, NLS, LSF, SFS, FSE, SEL, ELC, LCC, ...","[MSE, SED, EDR, DRH, RHQ, HQQ, QQR, QRQ, RQQ, ..."
6268,"[MAP, APE, PEI, EIN, INL, NLP, LPG, PGP, GPM, ...","[MKN, KNL, NLS, LSF, SFV, FVA, VAG, AGL, GLV, ..."


In [ ]:
data1,data2,data1_test,data2_test,num_words,MAX_SEQUENCE_LENGTH,MAX_VOCAB_SIZE = f.get_seq_data_doubleip(7000,1000,df_train,df_test, pad='pre')

MAX_VOCAB_SIZE is 7000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 8624
min sequences1_train length: 37
median sequences1_train length: 467
max word index sequences1_train: 6999
max sequences2_train length: 3704
min sequences2_train length: 29
median sequences2_train length: 329
max word index sequences2_train: 6999
Found 8000 unique tokens in tokenizer1.
Found 7998 unique tokens in tokenizer2.
pre padding
Shape of data1 tensor: (6270, 1000)
Shape of data2 tensor: (6270, 1000)
max test_sequences1 length: 5530
min test_sequences1 length: 37
median test_sequences1 length: 484
max test_sequences2 length: 3704
min test_sequences2 length: 29
median test_sequences2 length: 324
pre padding for test seq.
Shape of test_data1 tensor: (1514, 1000)
Shape of test_data2 tensor: (1514, 1000)
num_words is 7000


In [ ]:
EMBEDDING_DIM = 5
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 20
M_1D=10
DROP = 0.6

ip1 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP,True)
ip2 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP,True)

concatenator = Concatenate(axis=1)
x = concatenator([ip1.output, ip2.output])
x = Dense(128)(x)
#x = concatenator([x1.output, x2.output]) # output is N x 4M
#x = Dense(128, activation='relu')(x)
x = Dropout(DROP)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_doubleip = Model(inputs=[ip1.input,ip2.input], outputs=output)
#plot_model(model1D_CNN_doubleip, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
model_doubleip.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
trains = [data1,data2,data2,data1]
tests = [data1_test,data2_test,data2_test,data1_test]

model_doubleip.fit(trains, df_train['label'].values,batch_size=BATCH_SIZE, epochs=EPOCHS,validation_data=(tests,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model_doubleip.predict(tests)))

Epoch 1/20
49/49 [==============================] - 52s 539ms/step - loss: 0.6760 - accuracy: 0.5726 - val_loss: 0.6007 - val_accuracy: 0.6830
Epoch 2/20
49/49 [==============================] - 24s 487ms/step - loss: 0.6261 - accuracy: 0.6660 - val_loss: 0.5972 - val_accuracy: 0.6935
Epoch 3/20
49/49 [==============================] - 24s 488ms/step - loss: 0.6090 - accuracy: 0.6708 - val_loss: 0.6152 - val_accuracy: 0.6605
Epoch 4/20
49/49 [==============================] - 24s 491ms/step - loss: 0.6036 - accuracy: 0.6832 - val_loss: 0.5766 - val_accuracy: 0.6889
Epoch 5/20
49/49 [==============================] - 24s 490ms/step - loss: 0.5868 - accuracy: 0.6840 - val_loss: 0.5421 - val_accuracy: 0.7213
Epoch 6/20
49/49 [==============================] - 24s 491ms/step - loss: 0.5187 - accuracy: 0.7419 - val_loss: 0.5432 - val_accuracy: 0.7206
Epoch 7/20
49/49 [==============================] - 24s 490ms/step - loss: 0.4673 - accuracy: 0.7753 - val_loss: 0.5120 - val_accuracy: 0.7530

In [ ]:
df_train,df_test = load_data(4)
print('The data used will be:')
df_train[['Human','Yersinia']]

The data used will be:


,Human,Yersinia
0,"[MKDK, KDKQ, DKQK, KQKK, QKKK, KKKK, KKKE, KKE...","[MAKA, AKAS, KASR, ASRH, SRHN, RHNL, HNLS, NLS..."
1,"[MSWS, SWSG, WSGL, SGLL, GLLH, LLHG, LHGL, HGL...","[MQHV, QHVT, HVTG, VTGS, TGSK, GSKR, SKRR, KRR..."
2,"[MSLF, SLFD, LFDL, FDLF, DLFR, LFRG, FRGF, RGF...","[MAEL, AELP, ELPA, LPAK, PAKR, AKRR, KRRF, RRF..."
3,"[MAVA, AVAP, VAPR, APRL, PRLF, RLFG, LFGG, FGG...","[MRIF, RIFA, IFAI, FAIS, AISC, ISCS, SCSS, CSS..."
4,"[MSTI, STIQ, TIQS, IQSE, QSET, SETD, ETDC, TDC...","[MSYA, SYAF, YAFP, AFPG, FPGT, PGTF, GTFP, TFP..."
...,...,...
6265,"[MSYF, SYFG, YFGE, FGEH, GEHF, EHFW, HFWG, FWG...","[MITT, ITTD, TTDG, TDGN, DGNS, GNSA, NSAV, SAV..."
6266,"[MTVG, TVGK, VGKS, GKSS, KSSK, SSKM, SKML, KML...","[MSQP, SQPP, QPPF, PPFW, PFWQ, FWQQ, WQQK, QQK..."
6267,"[MNNL, NNLS, NLSF, LSFS, SFSE, FSEL, SELC, ELC...","[MSED, SEDR, EDRH, DRHQ, RHQQ, HQQR, QQRQ, QRQ..."
6268,"[MAPE, APEI, PEIN, EINL, INLP, NLPG, LPGP, PGP...","[MKNL, KNLS, NLSF, LSFV, SFVA, FVAG, VAGL, AGL..."


In [ ]:
 data1,data2,data1_test,data2_test,num_words,MAX_SEQUENCE_LENGTH,MAX_VOCAB_SIZE = f.get_seq_data_doubleip(100000,1000,df_train,df_test,pad = 'pre')

MAX_VOCAB_SIZE is 100000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 8063
min sequences1_train length: 34
median sequences1_train length: 440
max word index sequences1_train: 99999
max sequences2_train length: 3707
min sequences2_train length: 26
median sequences2_train length: 320
max word index sequences2_train: 99999
Found 155225 unique tokens in tokenizer1.
Found 134906 unique tokens in tokenizer2.
pre padding
Shape of data1 tensor: (6270, 1000)
Shape of data2 tensor: (6270, 1000)
max test_sequences1 length: 5224
min test_sequences1 length: 34
median test_sequences1 length: 449
max test_sequences2 length: 3707
min test_sequences2 length: 24
median test_sequences2 length: 308
pre padding for test seq.
Shape of test_data1 tensor: (1514, 1000)
Shape of test_data2 tensor: (1514, 1000)
num_words is 100000


In [ ]:
EMBEDDING_DIM = 10
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10
M_1D=10
DROP = 0.7

ip1 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP,True)
ip2 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP,True)

concatenator = Concatenate(axis=1)
x = concatenator([ip1.output, ip2.output])
x = Dense(128)(x)
#x = concatenator([x1.output, x2.output]) # output is N x 4M
#x = Dense(128, activation='relu')(x)
x = Dropout(DROP)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_doubleip = Model(inputs=[ip1.input,ip2.input], outputs=output)
#plot_model(model1D_CNN_doubleip, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
model_doubleip.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
trains = [data1,data2,data2,data1]
tests = [data1_test,data2_test,data2_test,data1_test]

model_doubleip.fit(trains, df_train['label'].values,batch_size=BATCH_SIZE, epochs=EPOCHS,validation_data=(tests,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model_doubleip.predict(tests)))

Epoch 1/10
49/49 [==============================] - 38s 568ms/step - loss: 0.6904 - accuracy: 0.5515 - val_loss: 0.6238 - val_accuracy: 0.6995
Epoch 2/10
49/49 [==============================] - 25s 519ms/step - loss: 0.6134 - accuracy: 0.6739 - val_loss: 0.6249 - val_accuracy: 0.6275
Epoch 3/10
49/49 [==============================] - 26s 523ms/step - loss: 0.5763 - accuracy: 0.7114 - val_loss: 0.5480 - val_accuracy: 0.7226
Epoch 4/10
49/49 [==============================] - 26s 523ms/step - loss: 0.4693 - accuracy: 0.7842 - val_loss: 0.4579 - val_accuracy: 0.7787
Epoch 5/10
49/49 [==============================] - 26s 523ms/step - loss: 0.3120 - accuracy: 0.8789 - val_loss: 0.4403 - val_accuracy: 0.8038
Epoch 6/10
49/49 [==============================] - 26s 524ms/step - loss: 0.2139 - accuracy: 0.9162 - val_loss: 0.5882 - val_accuracy: 0.7715
Epoch 7/10
49/49 [==============================] - 26s 529ms/step - loss: 0.1334 - accuracy: 0.9508 - val_loss: 0.5489 - val_accuracy: 0.8131

In [ ]:
df_train,df_test = load_data(5)
print('The data used will be:')
df_train[['Human','Yersinia']]

The data used will be:


,Human,Yersinia
0,"[MKDKQ, KDKQK, DKQKK, KQKKK, QKKKK, KKKKE, KKK...","[MAKAS, AKASR, KASRH, ASRHN, SRHNL, RHNLS, HNL..."
1,"[MSWSG, SWSGL, WSGLL, SGLLH, GLLHG, LLHGL, LHG...","[MQHVT, QHVTG, HVTGS, VTGSK, TGSKR, GSKRR, SKR..."
2,"[MSLFD, SLFDL, LFDLF, FDLFR, DLFRG, LFRGF, FRG...","[MAELP, AELPA, ELPAK, LPAKR, PAKRR, AKRRF, KRR..."
3,"[MAVAP, AVAPR, VAPRL, APRLF, PRLFG, RLFGG, LFG...","[MRIFA, RIFAI, IFAIS, FAISC, AISCS, ISCSS, SCS..."
4,"[MSTIQ, STIQS, TIQSE, IQSET, QSETD, SETDC, ETD...","[MSYAF, SYAFP, YAFPG, AFPGT, FPGTF, PGTFP, GTF..."
...,...,...
6265,"[MSYFG, SYFGE, YFGEH, FGEHF, GEHFW, EHFWG, HFW...","[MITTD, ITTDG, TTDGN, TDGNS, DGNSA, GNSAV, NSA..."
6266,"[MTVGK, TVGKS, VGKSS, GKSSK, KSSKM, SSKML, SKM...","[MSQPP, SQPPF, QPPFW, PPFWQ, PFWQQ, FWQQK, WQQ..."
6267,"[MNNLS, NNLSF, NLSFS, LSFSE, SFSEL, FSELC, SEL...","[MSEDR, SEDRH, EDRHQ, DRHQQ, RHQQR, HQQRQ, QQR..."
6268,"[MAPEI, APEIN, PEINL, EINLP, INLPG, NLPGP, LPG...","[MKNLS, KNLSF, NLSFV, LSFVA, SFVAG, FVAGL, VAG..."


In [ ]:
 data1,data2,data1_test,data2_test,num_words,MAX_SEQUENCE_LENGTH,MAX_VOCAB_SIZE = f.get_seq_data_doubleip(500000,1000,df_train,df_test,pad = 'pre')

MAX_VOCAB_SIZE is 500000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 5301
min sequences1_train length: 12
median sequences1_train length: 327
max word index sequences1_train: 499999
max sequences2_train length: 3706
min sequences2_train length: 9
median sequences2_train length: 307
max word index sequences2_train: 499999
Found 1330540 unique tokens in tokenizer1.
Found 639136 unique tokens in tokenizer2.
pre padding
Shape of data1 tensor: (6270, 1000)
Shape of data2 tensor: (6270, 1000)
max test_sequences1 length: 4674
min test_sequences1 length: 10
median test_sequences1 length: 271
max test_sequences2 length: 3706
min test_sequences2 length: 5
median test_sequences2 length: 244
pre padding for test seq.
Shape of test_data1 tensor: (1514, 1000)
Shape of test_data2 tensor: (1514, 1000)
num_words is 500000


In [ ]:
EMBEDDING_DIM = 15
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10
M_1D=10
DROP = 0.8

ip1 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP,True)
ip2 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP,True)

concatenator = Concatenate(axis=1)
x = concatenator([ip1.output, ip2.output])
x = Dense(128)(x)
#x = concatenator([x1.output, x2.output]) # output is N x 4M
#x = Dense(128, activation='relu')(x)
x = Dropout(DROP)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_doubleip = Model(inputs=[ip1.input,ip2.input], outputs=output)
#plot_model(model1D_CNN_doubleip, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
model_doubleip.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
trains = [data1,data2,data2,data1]
tests = [data1_test,data2_test,data2_test,data1_test]

model_doubleip.fit(trains, df_train['label'].values,batch_size=BATCH_SIZE, epochs=EPOCHS,validation_data=(tests,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model_doubleip.predict(tests)))

Epoch 1/10
49/49 [==============================] - 48s 759ms/step - loss: 0.7035 - accuracy: 0.5466 - val_loss: 0.6385 - val_accuracy: 0.7160
Epoch 2/10
49/49 [==============================] - 35s 712ms/step - loss: 0.6004 - accuracy: 0.6763 - val_loss: 0.5676 - val_accuracy: 0.7550
Epoch 3/10
49/49 [==============================] - 35s 712ms/step - loss: 0.5660 - accuracy: 0.7178 - val_loss: 0.5589 - val_accuracy: 0.7279
Epoch 4/10
49/49 [==============================] - 35s 715ms/step - loss: 0.4461 - accuracy: 0.7941 - val_loss: 0.4314 - val_accuracy: 0.8078
Epoch 5/10
49/49 [==============================] - 35s 713ms/step - loss: 0.2607 - accuracy: 0.9013 - val_loss: 0.3951 - val_accuracy: 0.8263
Epoch 6/10
49/49 [==============================] - 35s 713ms/step - loss: 0.1317 - accuracy: 0.9520 - val_loss: 0.4573 - val_accuracy: 0.8032
Epoch 7/10
49/49 [==============================] - 35s 714ms/step - loss: 0.0809 - accuracy: 0.9768 - val_loss: 0.5061 - val_accuracy: 0.8078

In [ ]:
df_train,df_test = load_data(6)
print('The data used will be:')
df_train[['Human','Yersinia']]

The data used will be:


,Human,Yersinia
0,"[MKDKQK, KDKQKK, DKQKKK, KQKKKK, QKKKKE, KKKKE...","[MAKASR, AKASRH, KASRHN, ASRHNL, SRHNLS, RHNLS..."
1,"[MSWSGL, SWSGLL, WSGLLH, SGLLHG, GLLHGL, LLHGL...","[MQHVTG, QHVTGS, HVTGSK, VTGSKR, TGSKRR, GSKRR..."
2,"[MSLFDL, SLFDLF, LFDLFR, FDLFRG, DLFRGF, LFRGF...","[MAELPA, AELPAK, ELPAKR, LPAKRR, PAKRRF, AKRRF..."
3,"[MAVAPR, AVAPRL, VAPRLF, APRLFG, PRLFGG, RLFGG...","[MRIFAI, RIFAIS, IFAISC, FAISCS, AISCSS, ISCSS..."
4,"[MSTIQS, STIQSE, TIQSET, IQSETD, QSETDC, SETDC...","[MSYAFP, SYAFPG, YAFPGT, AFPGTF, FPGTFP, PGTFP..."
...,...,...
6265,"[MSYFGE, SYFGEH, YFGEHF, FGEHFW, GEHFWG, EHFWG...","[MITTDG, ITTDGN, TTDGNS, TDGNSA, DGNSAV, GNSAV..."
6266,"[MTVGKS, TVGKSS, VGKSSK, GKSSKM, KSSKML, SSKML...","[MSQPPF, SQPPFW, QPPFWQ, PPFWQQ, PFWQQK, FWQQK..."
6267,"[MNNLSF, NNLSFS, NLSFSE, LSFSEL, SFSELC, FSELC...","[MSEDRH, SEDRHQ, EDRHQQ, DRHQQR, RHQQRQ, HQQRQ..."
6268,"[MAPEIN, APEINL, PEINLP, EINLPG, INLPGP, NLPGP...","[MKNLSF, KNLSFV, NLSFVA, LSFVAG, SFVAGL, FVAGL..."


In [ ]:
 data1,data2,data1_test,data2_test,num_words,MAX_SEQUENCE_LENGTH,MAX_VOCAB_SIZE = f.get_seq_data_doubleip(850000,1000,df_train,df_test,pad = 'pre')

MAX_VOCAB_SIZE is 850000
MAX_SEQUENCE_LENGTH is 1000
max sequences1_train length: 5630
min sequences1_train length: 0
median sequences1_train length: 199
max word index sequences1_train: 849999
max sequences2_train length: 3705
min sequences2_train length: 0
median sequences2_train length: 329
max word index sequences2_train: 849999
Found 2438322 unique tokens in tokenizer1.
Found 864366 unique tokens in tokenizer2.
pre padding
Shape of data1 tensor: (6270, 1000)
Shape of data2 tensor: (6270, 1000)
max test_sequences1 length: 5630
min test_sequences1 length: 0
median test_sequences1 length: 70
max test_sequences2 length: 3705
min test_sequences2 length: 0
median test_sequences2 length: 284
pre padding for test seq.
Shape of test_data1 tensor: (1514, 1000)
Shape of test_data2 tensor: (1514, 1000)
num_words is 850000


In [ ]:
EMBEDDING_DIM = 15
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10
M_1D=10
DROP = 0.8

ip1 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP,True)
ip2 = f.att_model(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,num_words,DROP,True)

concatenator = Concatenate(axis=1)
x = concatenator([ip1.output, ip2.output])
x = Dense(128)(x)
#x = concatenator([x1.output, x2.output]) # output is N x 4M
#x = Dense(128, activation='relu')(x)
x = Dropout(DROP)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_doubleip = Model(inputs=[ip1.input,ip2.input], outputs=output)
#plot_model(model1D_CNN_doubleip, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
model_doubleip.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
trains = [data1,data2,data2,data1]
tests = [data1_test,data2_test,data2_test,data1_test]

model_doubleip.fit(trains, df_train['label'].values,batch_size=BATCH_SIZE, epochs=EPOCHS,validation_data=(tests,df_test['label'].values))
print(roc_auc_score(df_test['label'].values, model_doubleip.predict(tests)))

Epoch 1/10
49/49 [==============================] - 59s 1s/step - loss: 0.6983 - accuracy: 0.5572 - val_loss: 0.6193 - val_accuracy: 0.7596
Epoch 2/10
49/49 [==============================] - 44s 909ms/step - loss: 0.6136 - accuracy: 0.6663 - val_loss: 0.5697 - val_accuracy: 0.7543
Epoch 3/10
49/49 [==============================] - 45s 913ms/step - loss: 0.5628 - accuracy: 0.7158 - val_loss: 0.5280 - val_accuracy: 0.7536
Epoch 4/10
49/49 [==============================] - 45s 912ms/step - loss: 0.4631 - accuracy: 0.7767 - val_loss: 0.4507 - val_accuracy: 0.8018
Epoch 5/10
49/49 [==============================] - 45s 910ms/step - loss: 0.2844 - accuracy: 0.8896 - val_loss: 0.4526 - val_accuracy: 0.8085
Epoch 6/10
49/49 [==============================] - 44s 905ms/step - loss: 0.2202 - accuracy: 0.9119 - val_loss: 0.5424 - val_accuracy: 0.8157
Epoch 7/10
49/49 [==============================] - 44s 906ms/step - loss: 0.1583 - accuracy: 0.9375 - val_loss: 0.6821 - val_accuracy: 0.7913
Ep